<a href="https://colab.research.google.com/github/fatemafaria142/Exploration-of-Large-Language-Models-in-Mental-Health-Advice-Generation/blob/main/Mental_Health_Advice_Generation_using_openchat_3_5_1210_and_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Required Packages**

In [ ]:
!pip install accelerate peft bitsandbytes transformers trl datasets torch langchain

#### **Dataset Link:** https://huggingface.co/datasets/Amod/mental_health_counseling_conversations

In [96]:
from datasets import load_dataset

# Load the dataset
instruct_tune_dataset = load_dataset("Amod/mental_health_counseling_conversations")

# Select 3000 rows for training
train_subset = instruct_tune_dataset["train"].select(indices=range(3000))

# Select the rest for testing
test_subset = instruct_tune_dataset["train"].select(indices=range(3000, len(instruct_tune_dataset["train"])))


In [97]:
train_subset

Dataset({
    features: ['Context', 'Response'],
    num_rows: 3000
})

In [98]:
test_subset

Dataset({
    features: ['Context', 'Response'],
    num_rows: 512
})

In [99]:
# Check the sizes of the subsets
print("Train subset size:", len(train_subset))
print("Test subset size:", len(test_subset))

Train subset size: 3000
Test subset size: 512


In [100]:
# Show 5 examples from the train dataset
print("Train subset examples:")
for i in range(5):
    context = train_subset['Context'][i]
    response = train_subset['Response'][i]
    print(f"Example {i + 1}:")
    print(f"Context: {context}")
    print(f"Response: {response}")
    print("-------------------")

Train subset examples:
Example 1:
Context: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone?
Response: If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media.  Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't feel good about themselves

In [101]:
# Show 5 examples from the test dataset
print("\nTest subset examples:")
for i in range(5):
    context = test_subset['Context'][i]
    response = test_subset['Response'][i]
    print(f"Example {i + 1}:")
    print(f"Context: {context}")
    print(f"Response: {response}")
    print("-------------------")


Test subset examples:
Example 1:
Context: I had to put a restraining order against my ex-fiancé. He was served last night. He was mentally and verbally abusive towards me. I thought it was my nerves, but every time I stand up, I get sick. If I sit and rock, I'm fine.
Response: Hello, and thank you for your question. You may be right, your physical symptoms of getting sick to your stomach may very well be your nerves or anxiety. That is totally normal and understandable given your circumstances. Another thing that is not unusual is that you feel better when you sit and rock. That is actually a fairly common way that many people comfort themselves in times of high stress. If you are concerned that there is an actual physical illness causing these symptoms, you may want to visit your primary care provider. They may tell you that they don't see anything physically wrong with you, and at that point you may want to visit a counselor about the symptoms and your relationship experience. A vis

#### **Creating a Mental Health Support Prompt Using LangChain**

In [117]:
from langchain import PromptTemplate, LLMChain

def create_prompt(sample):
    # Create a sample prompt template
    template = """GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: {context}<|end_of_turn|>
GPT4 Correct Assistant: {response}<|end_of_turn|>
"""

    # Extract context and response from the sample
    example_context = sample['Context']
    example_response = sample['Response']

    # Create a prompt using the template and the example
    prompt_template = PromptTemplate(template=template, input_variables=["context", "response"])
    single_turn_prompt = prompt_template.format(context=example_context, response=example_response)

    return single_turn_prompt

#### **Generating LangChain Prompt for a Training Subset Examples**

In [118]:
prompt = create_prompt(train_subset[0])
print(prompt)

GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone?<|end_of_turn|>
GPT4 Correct Assistant: If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same 

In [119]:
prompt = create_prompt(train_subset[1])
print(prompt)

GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone?<|end_of_turn|>
GPT4 Correct Assistant: Hello, and thank you for your question and seeking advice on this. Feelings of worthlessness is unfortunately common. In fact, most people, if not all, have felt this to some degree at some point in their life. You are not alone. Changing our feelings is like changing our thoughts - it's ha

In [121]:
prompt = create_prompt(train_subset[2])
print(prompt)

GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone?<|end_of_turn|>
GPT4 Correct Assistant: First thing I'd suggest is getting the sleep you need or it will impact how you think and feel. I'd look at finding what is going well in your life and what you can be grateful for. I believe everyone has talents and wants to find their purpose in life. I think you can figure it out with so

#### **Generating LangChain Prompt for a Testing Subset Examples**

In [122]:
prompt = create_prompt(test_subset[0])
print(prompt)

GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I had to put a restraining order against my ex-fiancé. He was served last night. He was mentally and verbally abusive towards me. I thought it was my nerves, but every time I stand up, I get sick. If I sit and rock, I'm fine.<|end_of_turn|>
GPT4 Correct Assistant: Hello, and thank you for your question. You may be right, your physical symptoms of getting sick to your stomach may very well be your nerves or anxiety. That is totally normal and understandable given your circumstances. Another thing that is not unusual is that you feel better when you sit and rock. That is actually a fairly common way that many people comfort themselves in times of high s

In [123]:
prompt = create_prompt(test_subset[1])
print(prompt)

GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I’m trying to make marriage work after a split. Before our split, he lied a lot and broke every promise to me. I don't think he cheated. Last month, I asked what women work with him, so he told me. Yesterday, I found out about a girl that he said he forgot about. Should I be upset?<|end_of_turn|>
GPT4 Correct Assistant: I would ask you first what made you give him another chance after he repeatedly lied and broke every promise to you?  I would imagine if he repeatedly lied to you that it will damage your ability to trust him now.  Is he in therapy? Does he recognize that he has a problem and is he trying to repair it?  Even if he truly forgot to tell 

In [124]:
prompt = create_prompt(test_subset[2])
print(prompt)

GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: My fiancé and I broke up. He cheated on me numerous times. I kept forgiving but questioning his every move. He got tired and left.<|end_of_turn|>
GPT4 Correct Assistant: if he as cheated on you multiple times it is not healthy for you to continue seeing him.  However It takes time to heal your pain. You are not a robot that can just switch off your emotions.  Please surround yourself with people who can support and empower you. <|end_of_turn|>



### **Initializing the Model**
* Load the model using a 4-bit configuration, employing double quantization, and set bfloat16 as the compute data type.

* Notably, we opt for the instruct-tuned model in this instance rather than the base model. It's worth mentioning that fine-tuning a base model necessitates a more substantial amount of data!

In [125]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )


* **Model and Tokenizer:** https://huggingface.co/openchat/openchat-3.5-1210

In [126]:
mode_id = "openchat/openchat-3.5-1210"

In [127]:
model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto", use_cache=False
    )

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [128]:
tokenizer = AutoTokenizer.from_pretrained("openchat/openchat-3.5-1210")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### **Let's example how well the model does at this task currently(without training):**
* `do_sample:`This parameter controls whether to use sampling during text generation. When do_sample is set to True, the model samples words for the next token based on their probabilities. If set to False, the model uses greedy decoding and selects the token with the highest probability at each step.
* `pad_token_id:` This parameter represents the token ID used for padding in the tokenizer. In text generation, it's common to set the padding token to the end-of-sequence token (eos_token_id) to ensure that the generated text is complete.
* `top_k:` top_k controls the number of highest probability options considered during sampling. For each token, the model will limit the sampling to the top k tokens with the highest probabilities.
* **High top_k Value:** More deterministic and focused generation. The
model considers a larger number of top probability options, resulting in conservative and predictable outputs.
* **Low top_k Value:** More randomness in sampling. The model restricts the number of top probability options, leading to diverse and creative text generation, but it may result in less coherent or focused output.

* `top_p:` top_p is used in nucleus sampling, where the model considers the cumulative probability mass of the top tokens and stops sampling when it reaches a predefined threshold. It helps in preventing the model from being overly verbose and encourages diversity in the generated text.
* **High top_p Value:** Promotes diverse and varied text generation by considering a larger range of probable tokens. It's good for creating exploratory and varied outputs.
* **Low top_p Value:** Adopts a stricter threshold for selecting tokens, resulting in more conservative and focused text generation. This may lead to more deterministic outputs closely tied to the most probable tokens.
* `temperature:` temperature controls the level of randomness in the sampling process. A higher temperature (e.g., 1.0) increases randomness, making the generated text more diverse. Lower values (e.g., 0.8) make the sampling more deterministic, with the model focusing on higher probability tokens.

**Documentation Link:**
* https://python.langchain.com/docs/integrations/llms/huggingface_pipelines


In [133]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import pipeline
from langchain.prompts import PromptTemplate

# Additional configuration parameters
do_sample = True
pad_token_id = tokenizer.eos_token_id
top_k = 50  # You can adjust this value based on your preference
top_p = 0.95  # You can adjust this value based on your preference
temperature = 0.8  # You can adjust this value based on your preference

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=1024, do_sample=do_sample, pad_token_id=pad_token_id, top_k=top_k, top_p=top_p, temperature=temperature)
hf = HuggingFacePipeline(pipeline=pipe)

# Create a sample prompt template

template = """GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: {context}<|end_of_turn|>
GPT4 Correct Assistant: """
prompt = PromptTemplate(template=template, input_variables=["context"])


chain = prompt | hf

context = "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone?"

print(chain.invoke({"context": context}))

İ firstly want to say that I'm sorry you're going through this tough time. It's brave of you to reach out and seek help. It's important to remember that you're not alone and there is always hope for change and growth.

Feeling worthless can be an overwhelming and isolating experience, but it's important to understand that these feelings are not a true reflection of your value. They are often a result of internalizing negative thoughts and beliefs about yourself. Here are some steps you can take to begin shifting your perspective:

1. Acknowledge your feelings: It's important to validate your feelings and acknowledge that they are real and can be painful. This can help you feel less alone and more understood.

2. Challenge negative thoughts: When you notice negative thoughts about yourself, try to challenge them by asking yourself if they are true, helpful, or necessary. Often, these thoughts are exaggerations or distortions of reality.

3. Seek professional help: A mental health counse

### **Setting up the Training**
we will be using the `huggingface` and the `peft` library!

In [134]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")


* we need to prepare the model to be trained in 4bit so we will use the  **`prepare_model_for_kbit_training`** function from peft




In [135]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# **Training Hyperparameters**
The choice of hyperparameters is contingent upon the desired training duration. Pay special attention to the following key factors:

* `num_train_epochs/max_steps:` Dictates the number of iterations over the data. Exercise caution, as an excessive number may lead to overfitting!

* `learning_rate:` Governs the convergence speed of the model. Adjust this parameter judiciously for optimal results.

In [138]:
from transformers import TrainingArguments
output_model= "openchat_instruct_generation"
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=1,
        max_steps=100,
        fp16=True,
)


### **Setting up the trainer**

`max_seq_length`: Context window size


In [139]:
from trl import SFTTrainer

max_seq_length = 1024

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=create_prompt,  # This will apply the create_prompt mapping to all training and test dataset
  args=training_arguments,
  train_dataset=train_subset,      # Pass train_subset as train_dataset
  eval_dataset=test_subset         # Pass test_subset as eval_dataset
)


### **Training starts here**

In [140]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,2.170900
20,1.866900
30,1.831000
40,1.804400
50,1.718300
60,1.654800
70,2.003100
80,1.761000
90,1.906200
100,1.651700


TrainOutput(global_step=100, training_loss=1.8368203163146972, metrics={'train_runtime': 1527.0437, 'train_samples_per_second': 0.262, 'train_steps_per_second': 0.065, 'total_flos': 1.7483553570816e+16, 'train_loss': 1.8368203163146972, 'epoch': 0.35})

### **Save the model**

In [141]:
trainer.save_model("openchat_instruct_generation")

In [142]:
merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:229: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


#### **Generate Answers After Training**

* `invoke:` call the chain on an input
* `batch:` call the chain on a list of inputs
#### **Documentation Link:**
* https://python.langchain.com/docs/expression_language/interface

In [148]:
pipe = pipeline("text-generation", model=merged_model, tokenizer=tokenizer, max_new_tokens=1024, do_sample=do_sample, pad_token_id=pad_token_id, top_k=top_k, top_p=top_p, temperature=temperature)
hf = HuggingFacePipeline(pipeline=pipe)
# Create a sample prompt template
template = """GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: {context}<|end_of_turn|>
GPT4 Correct Assistant: """
prompt = PromptTemplate(template=template, input_variables=["context"])

chain = prompt | hf

### **Using `Invoke`**

In [149]:
context = "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone?"
response = chain.invoke({"context": context})
print(response)

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



I'm glad you're reaching out for help. It can be really tough to deal with feeling worthless, and it sounds like you're struggling with some difficult emotions. First, it's important to know that you are not alone in feeling this way. Many people struggle with feelings of worthlessness at some point in their lives.

One thing that can help is to focus on what you're good at and what you're proud of. When we're feeling down, it's easy to focus on our weaknesses and forget about our strengths. Make a list of things you're good at and things you're proud of. This can help you to see yourself in a more positive light.

Another thing that can help is to talk to someone about your feelings. This could be a friend, family member, or mental health professional. Talking about your feelings can help you to process them and feel less overwhelmed.

Finally, consider seeking professional help. A mental health professional can help you to explore the root of your feelings of worthlessness and provi

### **Using `Batch`**

In [153]:
context_1 = "It was over 20 years ago, but the pain has resurfaced again now because I have started seeing her Facebook posts about how great her life is. I feel so angry. How can I handle this?"
context_2 = "My fiancé and I broke up. He cheated on me numerous times. I kept forgiving but questioning his every move. He got tired and left."
context_3 = "I've gone to a couple therapy sessions so far and still everytime I walk in I get nervous and shaky.  Is this normal? Should I still be feeling like this?"

contexts = [context_1, context_2, context_3]

# Invoke the LangChain pipeline for multiple contexts
responses = chain.batch([{"context": ctx} for ctx in contexts])

# Print the responses
for context, response in zip(contexts, responses):
    print(f"Context: {context}")
    print(f"Response: {response}")
    print("-------------------")


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Context: It was over 20 years ago, but the pain has resurfaced again now because I have started seeing her Facebook posts about how great her life is. I feel so angry. How can I handle this?
Response: 20 years ago you went through something painful, and it feels like it's happening again now because you're seeing her Facebook posts about how great her life is. It's natural to feel angry about this situation because the pain you experienced was a part of your life, and now it's coming back up.

It's important to recognize that people have different experiences in life, and comparing ourselves to others can lead to feelings of envy and resentment. It's helpful to remind ourselves that we can't control what other people do or say, but we can control our reactions to them.

When you find yourself feeling angry, try to pause and take a few deep breaths. This can help you calm down and think more clearly about the situation. You might also try writing down your feelings in a journal or talki